In [1]:
import sys
from pathlib import Path

root_dir = Path.cwd().parent  # Go one folder up from /notebooks
print("Root directory:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# testing
from src.data_extract.bronze_extractor.fsds_loader import load_fsds_from_zip
from src.data_extract.bronze_extractor.extractor_bs import extract_balance_sheets
from src.data_extract.bronze_extractor.extractor_is import extract_income_statements
from src.data_extract.bronze_extractor.extractor_cf import extract_cash_flows
from src.data_extract.gold_builder.builder_pipeline import gen_yqs, build_everything_for_zip
print("Imports successful!")

Root directory: /Users/agalyaayyadurai/Automated-Equity-Valuation
Imports successful!


### Filings Dataframe per Zip

In [2]:
from pathlib import Path
from src.data_extract.bronze_extractor.filings_index import build_filings_index

zip_path = Path("../data/raw/2025q2.zip")
out_path = Path("../data/bronze/filings/year_quarter=2025Q2/filings.parquet")

filings_df = build_filings_index(zip_path, out_path=out_path)

print(filings_df.shape)
print(filings_df['sic'].nunique())
print("Unique companies:", filings_df["cik"].nunique())
print("Form breakdown:\n", filings_df["form"].value_counts())

(1252, 9)


### Balance Sheet Dataframe per zip

In [5]:
from pathlib import Path
from src.data_extract.bronze_extractor.extractor_bs import extract_balance_sheets

zip_path = Path("../data/raw/2025q2.zip")
out_path = Path("../data/bronze/bs/year_quarter=2025Q2/bs.parquet")

# Run the extractor
bs_df = extract_balance_sheets(zip_path)

print("Shape:", bs_df.shape)               # e.g. (50000, 15)
print("Columns:", bs_df.columns.tolist())  # should list adsh, tag, ddate, value, etc.
print("\nUnique filings:", bs_df['adsh'].nunique())
print("Unique tags:", bs_df['tag'].nunique())
print(bs_df.head())

# Save output neatly
out_path.parent.mkdir(parents=True, exist_ok=True)
bs_df.to_parquet(out_path, index=False)
print(f"Saved {len(bs_df):,} rows to {out_path}")


Extracted 1,278,344 BS facts from 2025q2.zip | filings=7,006
Shape: (1278344, 17)
Columns: ['adsh', 'tag', 'version', 'ddate', 'qtrs', 'uom', 'coreg', 'value', 'cik', 'name', 'form', 'fy', 'fp', 'period', 'filed', 'sic', 'source_zip']

Unique filings: 7006
Unique tags: 10674
                   adsh                                   tag       version  \
0  0000002488-25-000047                AccountsPayableCurrent  us-gaap/2025   
1  0000002488-25-000047                         AssetsCurrent  us-gaap/2025   
2  0000002488-25-000047               CommonStockSharesIssued  us-gaap/2025   
3  0000002488-25-000047            DeferredIncomeTaxAssetsNet  us-gaap/2025   
4  0000002488-25-000047  IntangibleAssetsNetExcludingGoodwill  us-gaap/2025   

      ddate qtrs     uom coreg         value  cik name form   fy   fp period  \
0  20250331    0     USD   NaN  2.206000e+09  NaN  NaN  NaN  NaN  NaN    NaN   
1  20250331    0     USD   NaN  2.159500e+10  NaN  NaN  NaN  NaN  NaN    NaN   
2  202503

### Income Statement Dataframe per zip

In [6]:
from pathlib import Path
from src.ata_extract.bronze_extractor.extractor_is import extract_income_statements

zip_path = Path("../data/raw/2025q2.zip")
out_path = Path("../data/bronze/is/year_quarter=2025Q2/is.parquet")
is_df = extract_income_statements(zip_path)

print("Shape:", is_df.shape)
print("Unique filings:", is_df["adsh"].nunique())
print("Unique tags:", is_df["tag"].nunique())
print(is_df.head())


out_path.parent.mkdir(parents=True, exist_ok=True)
is_df.to_parquet(out_path, index=False)

Extracted 711,229 IS facts from 2025q2.zip | filings=7,002
Shape: (711229, 17)
Unique filings: 7002
Unique tags: 9728
                   adsh                                    tag       version  \
0  0000002488-25-000047             CostOfGoodsAndServicesSold  us-gaap/2025   
1  0000002488-25-000047  IncomeLossFromEquityMethodInvestments  us-gaap/2025   
2  0000002488-25-000047                IncomeTaxExpenseBenefit  us-gaap/2025   
3  0000002488-25-000047                        InterestExpense  us-gaap/2025   
4  0000002488-25-000047                    OperatingIncomeLoss  us-gaap/2025   

      ddate qtrs  uom coreg         value  cik name form   fy   fp period  \
0  20240331    1  USD   NaN  2.913000e+09  NaN  NaN  NaN  NaN  NaN    NaN   
1  20250331    1  USD   NaN  7.000000e+06  NaN  NaN  NaN  NaN  NaN    NaN   
2  20250331    1  USD   NaN  1.230000e+08  NaN  NaN  NaN  NaN  NaN    NaN   
3  20240331    1  USD   NaN  2.500000e+07  NaN  NaN  NaN  NaN  NaN    NaN   
4  20240331    1

### Cash Flow Statement Dataframe per zip

In [2]:
from pathlib import Path
from src.data_extract.bronze_extractor.extractor_cf import extract_cash_flows

zip_path = Path("../data/raw/2025q2.zip")
out_path = Path("../data/bronze/cf/year_quarter=2025Q2/cf.parquet")
cf_df = extract_cash_flows(zip_path)

print("Shape:", cf_df.shape)
print("Unique filings:", cf_df["adsh"].nunique())
print("Unique tags:", cf_df["tag"].nunique())
print(cf_df.head())


out_path.parent.mkdir(parents=True, exist_ok=True)
cf_df.to_parquet(out_path, index=False)
print(f"Saved {len(cf_df):,} rows to {out_path}")

Extracted 597,068 CF facts from 2025q2.zip | filings=6,954
Shape: (597068, 17)
Unique filings: 6954
Unique tags: 29467
                   adsh                                                tag  \
0  0000002488-25-000047  CashAndCashEquivalentsAtCarryingValueIncluding...   
1  0000002488-25-000047                                 InventoryWriteDown   
2  0000002488-25-000047         PaymentsToAcquirePropertyPlantAndEquipment   
3  0000002488-25-000047              PaymentsToAcquireShortTermInvestments   
4  0000002488-25-000047                 ProceedsFromIssuanceOfLongTermDebt   

        version     ddate qtrs  uom coreg         value  cik name form   fy  \
0  us-gaap/2025  20240331    0  USD   NaN  4.190000e+09  NaN  NaN  NaN  NaN   
1  us-gaap/2025  20250331    1  USD   NaN  0.000000e+00  NaN  NaN  NaN  NaN   
2  us-gaap/2025  20250331    1  USD   NaN  2.120000e+08  NaN  NaN  NaN  NaN   
3  us-gaap/2025  20240331    1  USD   NaN  4.330000e+08  NaN  NaN  NaN  NaN   
4  us-gaap/2025  

### SILVER - Shares dataframe per zip

In [2]:
from pathlib import Path

reload(tm)
from src.data_extract.silver_shares import build_silver_shares

zip_path = Path("../data/raw/2025q2.zip")
out = Path("../data/silver/shares/year_quarter=2025Q2/shares_wide.parquet")

shares_2025q2 = build_silver_shares(zip_path, out_path=out)
print(shares_2025q2.head())

                   adsh   cik                            name    fy    period  \
0  0000002488-25-000047  2488      ADVANCED MICRO DEVICES INC  2025  20250331   
1  0000002969-25-000027  2969  AIR PRODUCTS & CHEMICALS, INC.  2025  20250331   
2  0000003499-25-000009  3499                  ALEXANDERS INC  2025  20250331   
3  0000003545-25-000079  3545                     ALICO, INC.  2025  20250331   
4  0000003570-25-000049  3570             CHENIERE ENERGY INC  2025  20250331   

      filed  form   sic  CommonSharesAuth  CommonSharesIssuable  ...  \
0  20250507  10-Q  3674      2.250000e+09                   NaN  ...   
1  20250501  10-Q  2810               NaN                   NaN  ...   
2  20250505  10-Q  6798      1.000000e+07                   NaN  ...   
3  20250513  10-Q  0100      1.500000e+07                   NaN  ...   
4  20250508  10-Q  4924      4.800000e+08                   NaN  ...   

   TreasuryShares  TrustShares  UnitsAuth  UnitsIssued  UnitsOut  WASOBasic  \
0

### sugest tag map run

In [2]:
from pathlib import Path
from src.data_extract.tools.suggest_tag_map import suggest_from_zip, save_suggestions

sug = suggest_from_zip(Path("../data/raw/2025q2.zip"), top_n=60)
save_suggestions(sug, Path("../logs/coverage/suggestions_2025Q2"))


### SILVER - Extract Balance Sheet into a dataframe

In [2]:
from pathlib import Path
from src.data_extract.silver_transformer.transformer_bs import transform_balance_sheet_to_wide
from src.data_extract.config.tag_map_min import BS_NEW

# Example usage
zip_path = Path("../data/raw/2025q2.zip")
out_path = Path("../data/silver/bs/year_quarter=2025Q2/bs.parquet")

bs_df, unknown = transform_balance_sheet_to_wide(zip_path, tag_map=BS_NEW, out_path=out_path)

print(bs_df.shape)
print(bs_df.columns.to_list())
print(bs_df[['TotalAssets', 'TotalLiabilities', 'ShareholdersEquity']])
print(unknown["tag"].value_counts().head(25))

Extracted 1,278,344 BS facts from 2025q2.zip | filings=7,006
(1096, 38)
['adsh', 'cik', 'name', 'fy', 'filed', 'period', 'sic', 'APIC', 'AccountsPayable', 'AccountsReceivable', 'AccumulatedOCI', 'CashAndCashEquivalents', 'CommonStockParValue', 'CommonStockValue', 'Goodwill', 'Intangibles', 'Inventory', 'LeaseLiabilityCurrent', 'LeaseLiabilityNoncurrent', 'LeaseROUAsset', 'LongTermDebt', 'NoncontrollingInterest', 'OtherCurrentLiabilities', 'OtherNoncurrentLiabilities', 'PPandE', 'PreferredStockParValue', 'PreferredStockValue', 'RetainedEarnings', 'ShareholdersEquity', 'ShortTermDebt', 'ShortTermInvestments', 'TemporaryEquity', 'TotalAssets', 'TotalCurrentAssets', 'TotalCurrentLiabilities', 'TotalLiabilities', 'TotalNoncurrentAssets', 'TreasuryStock']
      TotalAssets  TotalLiabilities  ShareholdersEquity
130  7.767200e+07      1.065200e+07        5.314000e+07
194  8.175870e+08               NaN        5.088130e+08
716  1.859600e+10      8.521000e+09        1.007500e+10
119  1.082300e+1

In [6]:
import pandas as pd
# bs = bs_df.copy()

# # 1) Balance sheet identity: Assets ≈ Liabilities + Equity
# bs["liab_plus_equity"] = bs["TotalLiabilities"].fillna(0) + bs["ShareholdersEquity"].fillna(0)
# bs["bs_diff"] = (bs["TotalAssets"] - bs["liab_plus_equity"]).abs()
# bad_bs = bs[bs["bs_diff"] > 1e-3]  # tolerance; adjust for rounding
# print("Out-of-balance filings:", len(bad_bs))

# # 2) Debt composition sanity
# bs["TotalDebt_calc"] = bs["ShortTermDebt"].fillna(0) + bs["LongTermDebt"].fillna(0)

# # 3) Working capital & current ratio (handy for modeling)
# bs["WorkingCapital"] = bs["TotalCurrentAssets"].fillna(0) - bs["TotalCurrentLiabilities"].fillna(0)
# bs["CurrentRatio"] = bs["TotalCurrentAssets"] / bs["TotalCurrentLiabilities"]

# # 4) Intangibles share of assets (useful flag)
# bs["IntangiblesShare"] = (bs["Goodwill"].fillna(0) + bs["Intangibles"].fillna(0)) / bs["TotalAssets"]

# print(bs[["adsh","cik","fy","bs_diff","CurrentRatio","IntangiblesShare"]].head())

# bs_df["bs_diff"] = abs(bs_df["TotalAssets"] - (bs_df["TotalLiabilities"] + bs_df["ShareholdersEquity"]))
# (bs_df["bs_diff"] < 1000).mean()

bs = bs_df.copy()
bs["bs_diff"] = (bs["TotalAssets"] - (bs["TotalLiabilities"].fillna(0) + bs["ShareholdersEquity"].fillna(0) + + bs["TemporaryEquity"].fillna(0))).abs()
print("Out-of-balance (>$1k):", (bs["bs_diff"] > 1_000).sum())

balanced_ratio = (abs(bs["TotalAssets"] - (bs["TotalLiabilities"].fillna(0) + bs["ShareholdersEquity"].fillna(0) + bs["TemporaryEquity"].fillna(0))) < 1000).mean()
print(f"Balanced ratio: {balanced_ratio:.1%}")


Out-of-balance (>$1k): 823
Balanced ratio: 22.7%


In [5]:
dupes = bs_df.groupby(["cik", "fy"]).size().reset_index(name="count")
dupes[dupes["count"] > 1]

,cik,fy,count


### SILVER - Transform Income Statement into wide DF

In [3]:
from pathlib import Path
from src.data_extract.silver_transformer.transformer_is import transform_income_statement_to_wide

# zip_path = Path("../data/raw/2025q2.zip")
# out_path = Path("../data/silver/is/year_quarter=2025Q2/is.parquet")

# is_wide, is_unknown = transform_income_statement_to_wide(zip_path, out_path=out_path, return_unknown=True)
print(is_unknown["tag"].value_counts().head(25))
is_wide.shape, is_wide.head()


tag
EarningsPerShareDiluted                                                                   733
EarningsPerShareBasic                                                                     726
InvestmentIncomeInterest                                                                  185
NetIncomeLossAttributableToNoncontrollingInterest                                         170
ProfessionalFees                                                                          156
FairValueAdjustmentOfWarrants                                                             142
NetIncomeLossAvailableToCommonStockholdersBasic                                           122
RestructuringCharges                                                                      115
LaborAndRelatedExpense                                                                     92
AssetImpairmentCharges                                                                     81
IncomeLossFromContinuingOperationsPerBasicShare         

((658, 24),
                      adsh      cik                              name    fy  \
 97   0000950170-25-091151  1000045           OLD MARKET CAPITAL CORP  2024   
 89   0000950170-25-083705  1002047                      NETAPP, INC.  2025   
 303  0001437749-25-017409  1004989                  SPAR GROUP, INC.  2024   
 99   0001005229-25-000149  1005229            COLUMBUS MCKINNON CORP  2024   
 302  0001437749-25-014208  1008586  STREAMLINE HEALTH SOLUTIONS INC.  2024   
 
         filed    period   sic  ComprehensiveIncome  CostOfRevenue  \
 97   20250630  20250331  6153                  NaN            NaN   
 89   20250609  20250430  3572                  NaN   1.959000e+09   
 303  20250516  20241231  7389           -4412000.0   1.583570e+08   
 99   20250528  20250331  3531                  NaN   6.373470e+08   
 302  20250502  20250131  7373                  NaN   6.804000e+06   
 
      DebtExtinguishmentGainLoss  ...  IncomeTaxExpense  InterestExpense  \
 97           

In [5]:
isw = is_wide.copy()
# if your canon names are: Revenues, CostOfRevenue, GrossProfit, OperatingExpenses, OperatingIncomeLoss, NetIncomeLoss
isw["gross_calc"] = isw["Revenue"].fillna(0) - isw["CostOfRevenue"].fillna(0)
isw["gross_err"] = (isw["gross_calc"] - isw["GrossProfit"].fillna(0)).abs()
print("Median gross mismatch:", isw["gross_err"].median())

Median gross mismatch: 0.0


### SILVER - Transform Cash FLow into wide DF

In [6]:
from pathlib import Path
from src.data_extract.silver_transformer.transformer_cf import transform_cash_flow_to_wide

# zip_path = Path("../data/raw/2025q2.zip")
# out_path = Path("../data/silver/cf/year_quarter=2025Q2/cf.parquet")

# cf_wide, cf_unknown = transform_cash_flow_to_wide(zip_path, out_path=out_path, return_unknown=True)
print(cf_wide.columns.to_list())
cf_wide.shape, cf_wide.head()

['adsh', 'cik', 'name', 'fy', 'filed', 'period', 'sic', 'AcquireIntangibles', 'BusinessAcquisitions', 'CFF', 'CFI', 'CFO', 'CapEx', 'ChangeInAP', 'ChangeInAR', 'ChangeInAccruedLiabilities', 'ChangeInDeferredRevenue', 'ChangeInInventory', 'ChangeInPrepaidAndOther', 'DebtIssuance', 'DebtRepayment', 'DepAmortCF', 'EquityIssuance', 'FXEffect', 'IncomeTaxesPaid', 'InterestPaid', 'ProceedsFromSalePPE', 'ROUAssetNoncash', 'ShareRepurchase', 'StockBasedComp']


((657, 30),
                      adsh      cik                              name    fy  \
 98   0000950170-25-091151  1000045           OLD MARKET CAPITAL CORP  2024   
 90   0000950170-25-083705  1002047                      NETAPP, INC.  2025   
 304  0001437749-25-017409  1004989                  SPAR GROUP, INC.  2024   
 100  0001005229-25-000149  1005229            COLUMBUS MCKINNON CORP  2024   
 303  0001437749-25-014208  1008586  STREAMLINE HEALTH SOLUTIONS INC.  2024   
 
         filed    period   sic  AcquireIntangibles  BusinessAcquisitions  \
 98   20250630  20250331  6153                 NaN                   NaN   
 90   20250609  20250430  3572                 NaN                   0.0   
 304  20250516  20241231  7389                 NaN                   NaN   
 100  20250528  20250331  3531                 NaN                   0.0   
 303  20250502  20250131  7373                 NaN                   NaN   
 
              CFF  ...  DebtRepayment   DepAmortCF  Eq

In [7]:
cf = cf_wide.copy()

cf["calc_delta_cash"] = cf[["CFO", "CFI", "CFF"]].sum(axis=1, skipna=True)
cf["delta_cash_err"] = cf["calc_delta_cash"].abs()

print("Median |CFO+CFI+CFF| (proxy for ΔCash):", cf["delta_cash_err"].median())
cf[["CFO", "CFI", "CFF", "calc_delta_cash"]].head()

Median |CFO+CFI+CFF| (proxy for ΔCash): 982000.0


,CFO,CFI,CFF,calc_delta_cash
98,-1.900000e+06,12548000.0,-4801000.0,5847000.0
90,1.506000e+09,147000000.0,-828000000.0,825000000.0
304,-6.650000e+05,9881000.0,-1657000.0,7559000.0
100,4.561200e+07,-19891000.0,-86747000.0,-61026000.0
303,-1.514000e+06,-858000.0,1365000.0,-1007000.0


### SILVER - Shares Values from Financial Statements in a dataframe

In [2]:
from pathlib import Path
from src.data_extract.silver_transformer.transformer_shares import transform_shares_to_wide

zip_path = Path("../data/raw/2025q2.zip")
out_path = Path("../data/silver/shares/year_quarter=2025Q2/shares_wide.parquet")

shares_wide, shares_unknown = transform_shares_to_wide(zip_path, out_path=out_path, return_unknown=True)
shares_wide.shape, shares_wide.head()


((663, 31),
                      adsh      cik                              name    fy  \
 96   0000950170-25-091151  1000045           OLD MARKET CAPITAL CORP  2024   
 88   0000950170-25-083705  1002047                      NETAPP, INC.  2025   
 305  0001437749-25-017409  1004989                  SPAR GROUP, INC.  2024   
 98   0001005229-25-000149  1005229            COLUMBUS MCKINNON CORP  2024   
 304  0001437749-25-014208  1008586  STREAMLINE HEALTH SOLUTIONS INC.  2024   
 
         filed    period   sic  AntidilutiveExcluded  CommonSharesAuthorized  \
 96   20250630  20250331  6153                   NaN              50000000.0   
 88   20250609  20250430  3572                   NaN             885000000.0   
 305  20250516  20241231  7389                   NaN              47000000.0   
 98   20250528  20250331  3531                   NaN              50000000.0   
 304  20250502  20250131  7373                   NaN              85000000.0   
 
      CommonSharesIssued  ... 

In [3]:
sh = shares_wide.copy()

# 1) Shares must be positive
print("Share columns with negative values (should be ~0):")
share_cols = [c for c in sh.columns if c not in {"adsh","cik","name","fy","filed","period","sic"}]
neg_counts = {c: int((sh[c] < 0).sum()) for c in share_cols if sh[c].dtype != "O"}
{k:v for k,v in neg_counts.items() if v > 0}

# 2) Diluted >= Basic (when both exist)
if {"WASO_Basic","WASO_Diluted"}.issubset(set(share_cols)):
    ratio = (sh["WASO_Diluted"] >= sh["WASO_Basic"]).mean()
    print("Diluted >= Basic (% filings):", f"{ratio:.1%}")

# 3) Compare BS outstanding vs WA shares (not equal, but same order of magnitude)
cands = [c for c in share_cols if "Outstanding" in c or "WASO" in c]
sh[cands].describe().T.head(10)

Share columns with negative values (should be ~0):


,count,mean,std,min,25%,50%,75%,max
CommonSharesOutstanding,644.0,2.172613e+09,2.924156e+10,1.000000e+02,7.272570e+06,2.719911e+07,1.000258e+08,5.508343e+11
PreferredSharesOutstanding,373.0,5.576391e+06,6.662419e+07,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+05,1.248203e+09
UnitsOutstanding,4.0,3.453834e+07,6.500932e+07,4.628030e+04,1.199815e+06,3.047164e+06,3.638569e+07,1.320128e+08
WASOAdjustmentsDiluted,5.0,3.527003e+09,7.884126e+09,0.000000e+00,0.000000e+00,0.000000e+00,4.458169e+06,1.763055e+10
WASOBasic,547.0,2.176484e+09,3.091035e+10,4.078000e+03,6.900000e+06,2.589947e+07,9.813172e+07,5.408176e+11
WASOCombinedBasicDiluted,14.0,6.962515e+07,7.543315e+07,2.978540e+06,1.825181e+07,5.544130e+07,7.830784e+07,2.633375e+08
WASODiluted,547.0,2.113382e+09,3.089519e+10,0.000000e+00,7.028380e+06,2.741023e+07,1.007675e+08,5.408176e+11
WASOLimitedPartnership,3.0,7.336267e+07,5.667011e+07,1.914773e+07,4.394186e+07,6.873600e+07,1.004701e+08,1.322043e+08
WASOProFormaDiluted,1.0,7.426401e+09,NaN,7.426401e+09,7.426401e+09,7.426401e+09,7.426401e+09,7.426401e+09
WarrantsRightsOutstanding,1.0,4.899497e+06,NaN,4.899497e+06,4.899497e+06,4.899497e+06,4.899497e+06,4.899497e+06


### SILVER - Transform metadata dataframe

In [6]:
from pathlib import Path
from src.data_extract.silver_transformer.transformer_metadata import transform_metadata_to_wide

zip_path = Path("../data/raw/2025q2.zip")
out_path = Path("../data/silver/meta/year_quarter=2025Q2/meta.parquet")

meta_df = transform_metadata_to_wide(zip_path, out_path=out_path)
print(meta_df.head())
print(len(meta_df))

                     adsh      cik  \
0    0000950170-25-091151  1000045   
541  0001140361-25-016316  1000177   
644  0001140361-25-015768  1001290   
596  0001558370-25-005643  1001807   
133  0000950170-25-083705  1002047   

                                                  name  form    fy  fp  \
0                              OLD MARKET CAPITAL CORP  10-K  2024  FY   
541                        NORDIC AMERICAN TANKERS LTD  20-F  2024  FY   
644                                      CREDICORP LTD  20-F  2024  FY   
596  PERUSAHAAN PERSEROAN PERSERO PT TELEKOMUNIKASI...  20-F  2024  FY   
133                                       NETAPP, INC.  10-K  2025  FY   

       period     filed   sic industry  
0    20250331  20250630  6153  Unknown  
541  20241231  20250429  4400  Unknown  
644  20241231  20250425  6029  Unknown  
596  20241231  20250428  4813  Unknown  
133  20250430  20250609  3572  Unknown  
1218


### GOLD - Merged IS BS CF for one zip

In [3]:
from pathlib import Path
from src.data_extract.gold_builder.builder_per_zip import build_gold_zip
from src.data_extract.gold_builder.builder_all import build_gold_all
import pandas as pd

# 1) Build Gold for your current ZIP
yq = "2025Q2"
silver_dir = Path("../data/silver")
gold_dir = Path("../data/gold")

gold_z_path = build_gold_zip(yq, silver_dir, gold_dir)
print(gold_z_path)

# 2) Build the multi-year panel after you have more zips
panel_path = build_gold_all(gold_dir, gold_dir / "financials_panel.parquet")
panel = pd.read_parquet(panel_path)
panel.shape, panel.head()

../data/gold/2025Q2_financials.parquet


((1218, 87),
                    adsh      cik  \
 0  0000950170-25-091151  1000045   
 1  0001140361-25-016316  1000177   
 2  0001140361-25-015768  1001290   
 3  0001558370-25-005643  1001807   
 4  0000950170-25-083705  1002047   
 
                                                 name    fy     filed  \
 0                            OLD MARKET CAPITAL CORP  2024  20250630   
 1                        NORDIC AMERICAN TANKERS LTD  2024  20250429   
 2                                      CREDICORP LTD  2024  20250425   
 3  PERUSAHAAN PERSEROAN PERSERO PT TELEKOMUNIKASI...  2024  20250428   
 4                                       NETAPP, INC.  2025  20250609   
 
      period   sic industry source_zip  form  ... InterestPaid  \
 0  20250331  6153  Unknown     2025Q2  10-K  ...      47000.0   
 1  20241231  4400  Unknown     2025Q2  20-F  ...          NaN   
 2  20241231  6029  Unknown     2025Q2  20-F  ...          NaN   
 3  20241231  4813  Unknown     2025Q2  20-F  ...          

### Bronze + Silver + Gold - Run the zip files and add to the existing dataframe

In [6]:
from pathlib import Path
from src.data_extract.gold_builder.builder_pipeline import gen_yqs, build_everything_for_zip

#yqs = ["2025q1","2024q4", "2024q3", "2024q2", "2024q1", "2023q4", "2023q3"]
yqs = gen_yqs(("2014", 4), ("2009", 1))
#yqs = ["2025q2"]
raw_dir = Path("../data/raw")
silver_dir = Path("../data/silver")
gold_dir = Path("../data/gold")

for yq in yqs:
    print(build_everything_for_zip(yq, raw_dir, silver_dir, gold_dir))

Extracted 965,770 BS facts from 2014q4.zip | filings=7,697
Extracted 1,025,704 IS facts from 2014q4.zip | filings=7,705
Extracted 727,172 CF facts from 2014q4.zip | filings=7,639
../data/gold/2014q4_financials.parquet
Extracted 969,936 BS facts from 2014q3.zip | filings=7,774
Extracted 1,019,156 IS facts from 2014q3.zip | filings=7,777
Extracted 718,095 CF facts from 2014q3.zip | filings=7,708
../data/gold/2014q3_financials.parquet
Extracted 1,041,323 BS facts from 2014q2.zip | filings=8,533
Extracted 733,547 IS facts from 2014q2.zip | filings=8,538
Extracted 723,133 CF facts from 2014q2.zip | filings=8,477
../data/gold/2014q2_financials.parquet
Extracted 1,046,489 BS facts from 2014q1.zip | filings=7,150
Extracted 1,055,060 IS facts from 2014q1.zip | filings=7,154
Extracted 937,055 CF facts from 2014q1.zip | filings=7,086
../data/gold/2014q1_financials.parquet
Extracted 976,467 BS facts from 2013q4.zip | filings=7,835
Extracted 1,067,012 IS facts from 2013q4.zip | filings=7,836
Extrac

In [7]:
# After multiple zips are built:
from src.data_extract.gold_builder.builder_all import build_gold_all
panel_path = build_gold_all(gold_dir, gold_dir / "financials_panel.parquet")
print(panel_path)

../data/gold/financials_panel.parquet


In [8]:
from pathlib import Path
import pandas as pd

# Load the Gold panel
gold_file = Path("../data/gold/financials_panel.parquet")
df = pd.read_parquet(gold_file)

# Basic shape info
print(f"✅ Loaded {df.shape[0]:,} rows and {df.shape[1]} columns")

# 1️⃣ Total unique companies
unique_ciks = df["cik"].nunique()
print(f"🏢 Total unique CIKs (companies): {unique_ciks:,}")

# 2️⃣ Fiscal years per company
years_per_cik = df.groupby("cik")["fy"].nunique()

# 3️⃣ Companies with at least 2 unique fiscal years (≈ 2 years of 10-Ks)
multi_year_ciks = years_per_cik[years_per_cik >= 5]
print(f"📈 Companies with ≥ 2 fiscal years: {len(multi_year_ciks):,}")

# 4️⃣ Optional — inspect a few examples
print("\n🔍 Sample of multi-year companies:")
print(df[df["cik"].isin(multi_year_ciks.index)].sort_values(["cik", "fy"]).head(10))


✅ Loaded 365,495 rows and 71 columns
🏢 Total unique CIKs (companies): 14,550
📈 Companies with ≥ 2 fiscal years: 8,714

🔍 Sample of multi-year companies:
                       adsh      cik                    name  form    fy  fp  \
7695   0001193125-11-216128  1000045  NICHOLAS FINANCIAL INC  10-Q  2011  Q1   
14597  0001193125-11-303673  1000045  NICHOLAS FINANCIAL INC  10-Q  2011  Q2   
21746  0001193125-12-047920  1000045  NICHOLAS FINANCIAL INC  10-Q  2011  Q3   
28619  0001193125-12-270341  1000045  NICHOLAS FINANCIAL INC  10-K  2011  FY   
36943  0001193125-12-347645  1000045  NICHOLAS FINANCIAL INC  10-Q  2012  Q1   
44374  0001193125-12-462945  1000045  NICHOLAS FINANCIAL INC  10-Q  2012  Q2   
51849  0001193125-13-046001  1000045  NICHOLAS FINANCIAL INC  10-Q  2012  Q3   
58042  0001193125-13-259413  1000045  NICHOLAS FINANCIAL INC  10-K  2012  FY   
66124  0001193125-13-328101  1000045  NICHOLAS FINANCIAL INC  10-Q  2013  Q1   
73441  0001193125-13-438121  1000045  NICHOLAS 

## Filtering 10k only & Ticker, SIC mapping

In [11]:
from src.data_enrich.annual_financials import build_annual_10k_panel

annual_path = build_annual_10k_panel(
    panel_path="../data/gold/financials_panel.parquet",
    out_path="../data/enriched/financials_annual.parquet",
)
print("Annual panel saved to:", annual_path)


Annual panel saved to:                         adsh      cik                                name  \
12030   0001047469-11-006302     1750                            AAR CORP   
41736   0001047469-12-007300     1750                            AAR CORP   
70981   0001047469-13-007797     1750                            AAR CORP   
99717   0001047469-14-006243     1750                            AAR CORP   
127759  0001047469-15-006136     1750                            AAR CORP   
...                      ...      ...                                 ...   
358379  0001410578-25-000595  2039497  RESOLUTE HOLDINGS MANAGEMENT, INC.   
364141  0000950170-25-052639  2040127                KARMAN HOLDINGS INC.   
358380  0000950170-25-044778  2040807                       METSERA, INC.   
358381  0002042694-25-000003  2042694                   PRIMO BRANDS CORP   
364173  0002052959-25-000010  2052959             LIONSGATE STUDIOS CORP.   

        form    fy  fp    period      filed   sic so

In [13]:
from src.data_enrich.ticker_sic_mapping import build_financials_panel_ticker_sic

final_enriched_path = build_financials_panel_ticker_sic(
    parquet_in="../data/enriched/financials_annual.parquet",
    sic_mapping_path="../data/sic_office_industry.csv",
    parquet_ticker_intermediate="../data/enriched/financials_annual_ticker.parquet",
    parquet_out="../data/enriched/financials_annual_ticker_sic.parquet",
    save_intermediate=True,
)
print("Enriched Ticker + SIC panel saved to:", final_enriched_path)


[INFO] Saved intermediate panel with ticker to: ../data/enriched/financials_annual_ticker.parquet
[INFO] Saved enriched panel with ticker + SIC to: ../data/enriched/financials_annual_ticker_sic.parquet
Enriched Ticker + SIC panel saved to: ../data/enriched/financials_annual_ticker_sic.parquet


### Extracting OHLCV adn merging it to financials

In [8]:
import pandas as pd

# Loading the financials_with_ticker in a dataframe
df_funda = pd.read_parquet("../data/gold/financials_ticker_panel.parquet") 

from src.data_extract.bronze_extractor.extractor_ohlcv import make_ticker_period_ranges

ticker_ranges = make_ticker_period_ranges(df_funda, ticker_col="ticker", period_col="period")

ticker_ranges.to_parquet("../data/ticker_ranges.parquet")

In [ ]:
import pandas as pd

from src.data_extract.bronze_extractor.extractor_ohlcv import make_ticker_period_ranges, download_daily_prices_for_ranges

ticker_ranges = pd.read_parquet("../data/ticker_ranges.parquet")
# Download daily prices for all tickers and save as parquet
daily_prices = download_daily_prices_for_ranges(
    ticker_ranges=ticker_ranges,
    out_path="../data/daily_prices_1411.parquet",
    ticker_col="ticker",
    start_col="start_period",
    end_col="end_period",
    auto_adjust=False,   # or True if you prefer adjusted prices
    overwrite=True,      # set to False if you want to keep an existing file
)

daily_prices.head()



1 Failed download:
['AAAB']: YFPricesMissingError('possibly delisted; no price data found  (1d 2011-07-31 00:00:00 -> 2012-08-01 00:00:00)')
HTTP Error 404: 

1 Failed download:
['AAAO']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['AAC']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['AAMC']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['AAN']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['AAPC']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-12-31 00:00:00 -> 2020-01-01 00:00:00)')

1 Failed download:
['AAPH']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['AAQC']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['AARS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['AASL']: YFTzMissingError('possibly delisted; no timezone found')

1 Fa